# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
map_data = pd.read_csv('output.csv')
map_data

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Arraial do Cabo,-22.9661,-42.0278,72.10,84,100,7.67,BR
1,Griffith,41.5284,-87.4237,10.40,79,90,8.05,US
2,Kumul,42.8000,93.4500,33.22,72,99,5.53,CN
3,Hobart,-42.8794,147.3294,66.00,63,75,17.27,AU
4,Lebu,-37.6167,-73.6500,57.81,93,24,1.86,CL
...,...,...,...,...,...,...,...,...
553,Bella Vista,36.4295,-94.2316,30.99,93,90,19.57,US
554,Caconda,-13.7333,15.0667,62.49,65,100,5.21,AO
555,Ketchikan,55.3422,-131.6461,37.40,86,90,9.22,US
556,Hoi An,15.8794,108.3350,71.60,94,75,3.44,VN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = map_data[['Latitude', 'Longitude']].astype(float)
humidity_heatmap = map_data['Humidity'].astype(float)

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_heatmap, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
temp_narrow = map_data.loc[map_data['Max Temp'] >= 70]
temp_narrow2 = temp_narrow.loc[temp_narrow['Max Temp'] < 80]
wind_narrow = temp_narrow2.loc[temp_narrow2['Cloudiness'] == 0]
hotel_temp = wind_narrow.loc[wind_narrow['Wind Speed'] < 10]
hotel_df = hotel_temp.dropna()
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
46,Mahébourg,-20.4081,57.7000,75.00,86,0,1.99,MU
89,Shunde,22.8504,113.2470,77.00,69,0,4.45,CN
106,Hong Kong,22.2855,114.1577,73.00,70,0,1.01,HK
136,Upington,-28.4478,21.2561,73.96,38,0,8.97,ZA
137,Paraíso,9.8383,-83.8656,73.40,83,0,2.30,CR
187,Liuzhou,24.3126,109.3892,73.00,53,0,8.12,CN
191,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC
247,Manta,-0.9500,-80.7333,78.80,78,0,4.61,EC
251,Tabou,4.4230,-7.3528,78.69,89,0,8.16,CI
305,Te Anau,-45.4167,167.7167,75.51,39,0,6.08,NZ


In [8]:
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() + ['Hotel Name'])


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
#     base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [10]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
46,Mahébourg,-20.4081,57.7000,75.00,86,0,1.99,MU,Heritage Le Telfair Golf & Wellness Resort
89,Shunde,22.8504,113.2470,77.00,69,0,4.45,CN,"Langham Place, Guangzhou"
106,Hong Kong,22.2855,114.1577,73.00,70,0,1.01,HK,The OTTO Hotel
136,Upington,-28.4478,21.2561,73.96,38,0,8.97,ZA,Oranjerus Resort Upington
137,Paraíso,9.8383,-83.8656,73.40,83,0,2.30,CR,Hotel Costa Verde Inn
187,Liuzhou,24.3126,109.3892,73.00,53,0,8.12,CN,Lotus Villa Hotel
191,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC,Nataiwatch Guest House
247,Manta,-0.9500,-80.7333,78.80,78,0,4.61,EC,Hotel Vistalmar
251,Tabou,4.4230,-7.3528,78.69,89,0,8.16,CI,La Baie des Sirenes
305,Te Anau,-45.4167,167.7167,75.51,39,0,6.08,NZ,Lakefront Lodge Te Anau


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
locations = hotel_df[["Latitude", "Longitude"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)

In [17]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=89,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
hotels_list = hotel_df["Hotel Name"].tolist()

In [15]:
hotel_markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel: {hotel_info}" for hotel in hotels_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

fig


Figure(layout=FigureLayout(height='420px'))